In [91]:
from skimage.external.tifffile import TiffFile
import numpy as np
import os
import pandas as pd
import re
import napari

In [89]:
#use memmap to load from disk
with TiffFile('../test.tif') as tif:  
    data = tif.asarray(memmap = True)

In [92]:
#look at the data
viewer = napari.view_image(data, rgb=False)